# RNN Trainer

### Setup and Vars

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import os
import shutil

tfds.disable_progress_bar()

In [26]:
BUFFER_SIZE = 10000
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42 # for reproducibility

In [27]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

### Download Dataset

In [30]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [31]:

# 20000 reviews for train
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 5000 reviews for validation
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 25000 reviews for test
test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

print(class_names)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
['neg', 'pos']


In [43]:
#for example, label in train_dataset.take(1):
  #print('text: ', example.numpy())
  #print('label: ', label.numpy())

### Vocab Encoder for Word Tokenization

In [33]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [34]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 10,  69,  68, ...,   0,   0,   0],
       [  1,   2,   1, ...,   0,   0,   0],
       [ 10,  90, 389, ...,   0,   0,   0]], dtype=int64)

### Define Model

In [35]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [36]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [37]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]).astype(object))
print(predictions[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
[0.00498364]


In [38]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

### Train and Save Model

In [39]:
history = model.fit(train_dataset, epochs=1,
                    validation_data=val_ds,
                    validation_steps=30)

391/391 ━━━━━━━━━━━━━━━━━━━━ 210s 530ms/step - accuracy: 0.5057 - loss: 0.6838 - val_accuracy: 0.7604 - val_loss: 0.5056


In [40]:
test_loss, test_acc = model.evaluate(test_ds)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.7727 - loss: 0.5031
Test Loss: 0.5050932765007019
Test Accuracy: 0.7711600065231323


In [41]:
model.save_weights('Models/model1.weights.h5')

In [42]:
model.load_weights('Models/model1.weights.h5')